In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import torch
import ttach as tta
#from models import *
from resnest101 import *
from se_resnext101 import *
import pytorch_lightning as pl
from torch.nn import functional as F
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn import metrics
from config import *
from util import *
import optuna
import joblib
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
rc = {"figure.figsize" : (9, 7),
      "axes.spines.left" : False,
      "axes.spines.right" : False,
      "axes.spines.bottom" : False,
      "axes.spines.top" : False,
      "xtick.bottom" : True,
      "xtick.labelbottom" : True,
      "ytick.labelleft" : True,
      "ytick.left" : True,
      "axes.grid" : False}
plt.rcParams.update(rc)

def confidence_interval_accuracy(data, size=10000):
    """creates a bootstrap sample, computes replicates and returns replicates array"""
    # Create an empty array to store replicates
    bs_replicates = np.empty(size)
    
    np.random.seed(42)
    # Create bootstrap replicates as much as size
    for i in range(size):
        # Create a bootstrap sample
        bs_sample = data.sample(len(data), replace=True)
        # Get bootstrap replicate and append to bs_replicates
        bs_replicates[i] = metrics.balanced_accuracy_score(bs_sample['true'], bs_sample['pred'])
    
    return np.percentile(bs_replicates, [2.5, 97.5])

ModuleNotFoundError: No module named 'albumentations'

In [2]:
model_names = ['efficientnetb3', 'efficientnetb4.1', 'efficientnetb4.2', 'efficientnetb4.3', 'efficientnetb4.4', 'efficientnetb4.5', 
               'efficientnetb5.1', 'efficientnetb5.2', 'efficientnetb5.3', 'efficientnetb5.4', 'efficientnetb6.1', 'efficientnetb6.1', 
               'efficientnetb6.2', 'efficientnetb6.3', 'efficientnetb7.2', 'efficientnetb7.3', 'resnest101', 'se_resnext101', ]

#model_names = ['efficientnetb3', 'resnest101', 'se_resnext101']
dfs = []
for model_name in model_names:
    try:
        df = pd.read_csv(os.path.join('results', model_name+'.csv'))
        df['model'] = model_name
        dfs.append(df)
    except:
        pass
df = pd.concat(dfs)

In [5]:
result_ensemble = pd.read_csv(os.path.join('results', 'resnest101.csv'))

result_ensemble['score'] = df.reset_index().groupby('index').score.mean()
result_ensemble['pred'] = result_ensemble['score'].apply(lambda x: 1 if x >= 0.5 else 0)

display_scores(result_ensemble)

=====
AUC: 0.8433
Balanced Acc: 0.8
Sensitivity: 0.73
Specificity: 0.87
=====



In [4]:
confidence_interval_accuracy(result_ensemble)

array([0.74251211, 0.85296474])

In [36]:
for model in df.model.unique():
    temp = df[df.model == model]
    print(model, metrics.balanced_accuracy_score(temp.true, temp.pred))

efficientnetb3 0.74
efficientnetb4.1 0.755
efficientnetb4.2 0.72
efficientnetb4.3 0.72
efficientnetb4.4 0.72
efficientnetb4.5 0.72
efficientnetb5.1 0.78
efficientnetb5.2 0.725
efficientnetb5.3 0.725
efficientnetb5.4 0.7150000000000001
efficientnetb6.1 0.735
efficientnetb6.2 0.74
efficientnetb6.3 0.735
efficientnetb7.2 0.7150000000000001
efficientnetb7.3 0.695
resnest101 0.76
se_resnext101 0.775
